In [1]:
from fairworkflows import FairWorkflow, FairStep, FairVariable, is_fairworkflow
import inspect
import typing
from typing import Callable, get_type_hints, List, Union
import functools
from noodles import schedule

def _extract_inputs_from_function(func) -> List[FairVariable]:
    """
    Extract inputs from function using inspection. The name of the argument will be the name of
    the fair variable, the corresponding type hint will be the type of the variable.
    """
    argspec = inspect.getfullargspec(func)
    try:
        return [FairVariable(name=arg, type=argspec.annotations[arg].__name__)
                for arg in argspec.args]
    except KeyError:
        raise ValueError('Not all input arguments have type hinting, '
                         'FAIR step functions MUST have type hinting, '
                         'see https://docs.python.org/3/library/typing.html')
        
def _extract_outputs_from_function(func) -> List[FairVariable]:
    """
    Extract outputs from function using inspection. The name will be {function_name}_output{
    output_number}. The corresponding return type hint will be the type of the variable.
    """
    annotations = get_type_hints(func)
    try:
        return_annotation = annotations['return']
    except KeyError:
        raise ValueError('The return of the function does not have type hinting, '
                         'FAIR step functions MUST have type hinting, '
                         'see https://docs.python.org/3/library/typing.html')
    if _is_generic_tuple(return_annotation):
        return [FairVariable(name=func.__name__ + '_output' + str(i + 1), type=annotation.__name__)
                for i, annotation in enumerate(return_annotation.__args__)]
    else:
        return [FairVariable(name=func.__name__ + '_output1', type=return_annotation.__name__)]

def _is_generic_tuple(type_):
    """
    Check whether a type annotation is Tuple
    """
    if hasattr(typing, '_GenericAlias'):
        # 3.7
        # _GenericAlias cannot be imported from typing, because it doesn't
        # exist in all versions, and it will fail the type check in those
        # versions as well, so we ignore it.
        return (isinstance(type_, typing._GenericAlias)
                and type_.__origin__ is tuple)
    else:
        # 3.6 and earlier
        # GenericMeta cannot be imported from typing, because it doesn't
        # exist in all versions, and it will fail the type check in those
        # versions as well, so we ignore it.
        return (isinstance(type_, typing.GenericMeta)
                and type_.__origin__ is typing.Tuple)

    
    
    
def fairstep(func):

    label=func.__name__
    is_pplan_step = True
    is_manual_task = False
    is_script_task = True
    
    # Description of step is the raw function code
    description = inspect.getsource(func)
    inputs = _extract_inputs_from_function(func)
    outputs = _extract_outputs_from_function(func)
    func._fairstep = FairStep(label=label,
                                  description=description,
                                  is_pplan_step=is_pplan_step,
                                  is_manual_task=is_manual_task,
                                  is_script_task=is_script_task,
                                  inputs=inputs,
                                  outputs=outputs)
    
    return schedule(func)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>


# FairWorkflows execution demo

## Define the steps of your workflow
Each step should be its own function. Mark the function as such with the @fairstep decorator.

In [2]:
@fairstep
def add(a:float, b:float) -> float:
    """Adding up numbers!"""
    return a + b

In [3]:
@fairstep
def sub(a: float, b: float) -> float:
    """Subtracting numbers."""
    return a - b

In [4]:
@fairstep
def mul(a: float, b: float) -> float:
    """Multiplying numbers."""
    return a * b

In [5]:
@fairstep
def weird(a: float, b:float) -> float:
    """A weird function"""
    return a * 2 + b * 4
    

## Define your workflow using @fairworkflow
Now write a function which describes your workflow. Mark this function with the @fairworkflow decorator.

In [6]:
@is_fairworkflow(label='My Workflow')
def my_workflow(in1, in2, in3):
    """
    A simple addition, subtraction, multiplication workflow
    """
    t1 = add(in1, in2)
    t2 = sub(in1, in2)
    t3 = mul(weird(t1, in3), t2)
    return t3

## Create an instance of your workflow and display it

In [7]:
fw = my_workflow(1, 4, 3)
type(fw)

fairworkflows.fairworkflow.FairWorkflow

In [8]:
fw.display()

| workflow |
| --- |
| ![workflow workflow](control-workflow.svg) |

## Execute your workflow using .execute()
Set num_threads greater than 1 if you wish to exploit parallelisation in your workflow.

In [9]:
answer = fw.execute(num_threads=2)
answer

-66

## Get retrospective provenance of run
This is not in RDF format yet, but we have all the information needed.

In [10]:
fw.prov()

AttributeError: 'FairWorkflow' object has no attribute 'prov'

## Get Plex RDF of workflow

In [11]:
fw.get_workflow()

AttributeError: 'FairWorkflow' object has no attribute 'get_workflow'